In [3]:
import pandas as pd
import numpy as np

In [4]:
crime_data_2021_present = pd.read_csv('csv_files/Crimes_2021_to_Present.csv')

ML - 1. Predict if a person would get arrested considering primary type of crime, location description and neighborhood (region name)
2. Type of crime thats more prominent based on the history of the crimes in the neighborhood
3. Average price of the house based on the crimes (if lake shore had 1000 crimes, whats the future average price based on the crimes that has happened so far)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Feature Encoding
encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(crime_data_2021_present[['Primary Type', 'Location Description', 'RegionName']])
X_train, X_test, y_train, y_test = train_test_split(X_encoded, crime_data_2021_present['Arrest'], test_size=0.2, random_state=42)
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.911952871298121


/Users/lakshmikrishnan/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd 



# encoder = OneHotEncoder()

X = crime_data_2021_present[['Primary Type', 'Location Description', 'RegionName']]
y = crime_data_2021_present['Arrest'] 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

k_folds = KFold(n_splits=4, shuffle=True, random_state=42)
combination = [['Primary Type'], ['Primary Type', 'Location Description'], ['Primary Type', 'Location Description', 'RegionName'], ['Location Description'],['Location Description','RegionName'],['RegionName'], ['Primary Type','RegionName' ]]
avg_accuracy = {}

for features in combination:
    X_encoded = encoder.fit_transform(X_train[features])
    accuracies = []
    
    for train_idx, val_idx in k_folds.split(X_encoded):
        X_train_fold, X_val_fold = X_encoded[train_idx], X_encoded[val_idx] #Iterates over each combination, encoding the features, and perform k-fold cross-validation
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        model = LogisticRegression(random_state=42, solver='sag', penalty='l2', C=1.0)  
        model.fit(X_train_fold, y_train_fold)
        
        y_pred_val = model.predict(X_val_fold)
        accuracy = accuracy_score(y_val_fold, y_pred_val)
        accuracies.append(accuracy)
    
    avg_accuracy[str(features)] = sum(accuracies) / len(accuracies)

best_combination = max(avg_accuracy, key=avg_accuracy.get)
best_accuracy = avg_accuracy[best_combination]

print("Average accuracies:", avg_accuracy)
print("Best combination of features:", best_combination)
print("Best accuracy:", best_accuracy)


Average accuracies: {"['Primary Type']": 0.909402684217322, "['Primary Type', 'Location Description']": 0.9110558686390963, "['Primary Type', 'Location Description', 'RegionName']": 0.9112935253071257, "['Location Description']": 0.8799801532478356, "['Location Description', 'RegionName']": 0.8798621924993757, "['RegionName']": 0.8798864786236226, "['Primary Type', 'RegionName']": 0.9095605435855767}
Best combination of features: ['Primary Type', 'Location Description', 'RegionName']
Best accuracy: 0.9112935253071257


In [ ]:
# Highly Recommended for Final report ()
# 2 data (Crime and Housing)

# You want to get a mean house price of all the region in Housing data (181 columns and 1 row) 
# Rotate teh data (2 columns)
# Zone for different housing prices (Ex 10-50,000 Zone 1, 50-100,000 Zone 2, so on)
# Join them (Housing Data and Crime) on Region Name  => New Data (Save CSV)
# Create a ML Model Classificatio (SVM, NN)
# Using the Zone, Region Name, Primary Type, Location Description => Will the person be arrested or not (0,1)


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

encoder = OneHotEncoder()
X_encoded = encoder.fit_transform(crime_data_2021_present[['Primary Type', 'Location Description', 'RegionName']])
X_train, X_test, y_train, y_test = train_test_split(X_encoded, crime_data_2021_present['Arrest'], test_size=0.2, random_state=42)
print(X_train)
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
final_model = LogisticRegression(random_state=42, solver='liblinear', penalty='l2', C=1.0)
final_model.fit(X_encoded, y)
y_pred_test = final_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print("Test Accuracy:", test_accuracy)


  (0, 2)	1.0
  (0, 48)	1.0
  (0, 202)	1.0
  (1, 17)	1.0
  (1, 159)	1.0
  (1, 211)	1.0
  (2, 16)	1.0
  (2, 160)	1.0
  (2, 186)	1.0
  (3, 16)	1.0
  (3, 163)	1.0
  (3, 184)	1.0
  (4, 2)	1.0
  (4, 48)	1.0
  (4, 196)	1.0
  (5, 22)	1.0
  (5, 48)	1.0
  (5, 227)	1.0
  (6, 29)	1.0
  (6, 113)	1.0
  (6, 205)	1.0
  (7, 27)	1.0
  (7, 145)	1.0
  (7, 187)	1.0
  (8, 5)	1.0
  :	:
  (576454, 202)	1.0
  (576455, 5)	1.0
  (576455, 48)	1.0
  (576455, 179)	1.0
  (576456, 1)	1.0
  (576456, 48)	1.0
  (576456, 184)	1.0
  (576457, 22)	1.0
  (576457, 48)	1.0
  (576457, 204)	1.0
  (576458, 1)	1.0
  (576458, 145)	1.0
  (576458, 249)	1.0
  (576459, 2)	1.0
  (576459, 48)	1.0
  (576459, 208)	1.0
  (576460, 8)	1.0
  (576460, 145)	1.0
  (576460, 233)	1.0
  (576461, 3)	1.0
  (576461, 56)	1.0
  (576461, 217)	1.0
  (576462, 2)	1.0
  (576462, 48)	1.0
  (576462, 181)	1.0


/Users/lakshmikrishnan/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.911952871298121
Test Accuracy: 0.9121540980876516


In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


print(len(crime_data_2021_present['RegionName']))
print(len(y_test))
print(len(y_pred_test))

arrests_df = pd.DataFrame({'RegionName': crime_data_2021_present['RegionName'], 
                           'Actual': y_test, 
                           'Predicted': y_pred_test})


arrests_grouped = arrests_df.groupby('RegionName').sum().reset_index()
plt.figure(figsize=(10, 6))
plt.bar(arrests_grouped['RegionName'], arrests_grouped['Actual'], width=0.4, label='Actual')
plt.bar(arrests_grouped['RegionName'], arrests_grouped['Predicted'], width=0.4, label='Predicted', alpha=0.5)
plt.xlabel('Region Name')
plt.ylabel('Count')
plt.title('Actual vs Predicted Arrests by Region Name')
plt.xticks(rotation=90)
plt.legend()
plt.tight_layout()
plt.show()


ValueError: array length 144116 does not match index length 720579